# Лабораторная работа 2. Классификация текстов на основе вхождения в документ словарных слов

**Задание 1.** Загрузите в датафрейм новостной датасет `lenta_ru_news_filtered.csv`, собранный на базе корпуса `lenta.ru v1.0`. В датасете каждая новость описывается следующими полями:
* **url** - адрес новости на сайте `lenta.ru`,
* **topic** - тема новости,
* **title** - заголовок новости,
* **text** - текст новости.

Ответьте на следуюшие вопросы:
1. Сколько всего новостных текстов?
2. На какие темы встречаются новости?
3. Сколько новостных текстов в каждой теме?

In [ ]:
import pandas as pd

df = pd.read_csv('../../labs/lenta_ru_news_filtered.csv')

total_news = len(df)
print(f"1. Всего новостных текстов: {total_news}")

topics = df['topic'].unique()
print(f"\n2. Темы новостей:\n{topics}")

news_per_topic = df['topic'].value_counts()
print(f"\n3. Количество новостей по темам:\n{news_per_topic}")

**Задание 2.** Выполните предобработку новостных текстов в виде:
- приведение к нижнему регистру,
- удаление знаков пунктуации.

**(!) Далее в лабораторной работе задания выполняются с полученными обработанными текстами**.

Разделите датасет на обучающую и тестовую части в соотношении 80% к 20%. Выведите диаграммы, отражающие количество текстов по каждой теме в каждой из частей.

In [ ]:
import pandas as pd
import string
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

df['processed_text'] = df['text'].apply(preprocess_text)

print("Пример исходного текста:")
print(df['text'].iloc[0][:200])
print("\nПример обработанного текста:")
print(df['processed_text'].iloc[0][:200])

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['topic'])

print(f"\nРазмер обучающей выборки: {len(train_df)}")
print(f"Размер тестовой выборки: {len(test_df)}")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

train_topic_counts = train_df['topic'].value_counts()
ax1.bar(train_topic_counts.index, train_topic_counts.values)
ax1.set_title('Распределение тем в обучающей выборке')
ax1.set_xlabel('Темы')
ax1.set_ylabel('Количество новостей')
ax1.tick_params(axis='x', rotation=45)

test_topic_counts = test_df['topic'].value_counts()
ax2.bar(test_topic_counts.index, test_topic_counts.values)
ax2.set_title('Распределение тем в тестовой выборке')
ax2.set_xlabel('Темы')
ax2.set_ylabel('Количество новостей')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Детальная статистика по распределению
print("\nРаспределение по темам в обучающей выборке:")
print(train_df['topic'].value_counts())
print("\nРаспределение по темам в тестовой выборке:")
print(test_df['topic'].value_counts())

In [ ]:
train_df

**Задание 3.** Подсчитайте частоту встречаемости слов предобработанных новостных текстов `обучающей` части датафрейма. Какие слова употребляются наиболее часто вцелом в этих новостных текстах, а какие слова употребляются в этих же новостных текстах относительно тем (выведите топ-`50` слов для каждого случая)?

Нахождение частот слов в новостных текстах датафрейма можно выполнять посредством инструмента `FreqDist` библиотеки `NLTK`.

Например:
```
df['text'].apply(lambda x: nltk.FreqDist(nltk.word_tokenize(x)))
```

In [ ]:
import nltk
from nltk import FreqDist
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import pandas as pd

# Скачиваем необходимые ресурсы NLTK (если еще не скачаны)
nltk.download('punkt')

# Используем обучающую выборку из задания 2
print("Подсчет частоты слов в обучающей выборке...")

# 1. Частота слов во всей обучающей выборке
all_words = []
for text in train_df['processed_text']:
    tokens = word_tokenize(text)
    all_words.extend(tokens)

# Создаем объект FreqDist для всех слов
freq_dist_all = FreqDist(all_words)

# Выводим топ-50 самых частых слов
print("Топ-50 самых частых слов во всей обучающей выборке:")
top_50_all = freq_dist_all.most_common(50)
for i, (word, freq) in enumerate(top_50_all, 1):
    print(f"{i:2d}. {word}: {freq}")

# Визуализация топ-50 слов всей выборки
plt.figure(figsize=(12, 8))
words, frequencies = zip(*top_50_all)
plt.bar(range(len(words)), frequencies)
plt.xticks(range(len(words)), words, rotation=45, ha='right')
plt.title('Топ-50 самых частых слов в обучающей выборке')
plt.xlabel('Слова')
plt.ylabel('Частота')
plt.tight_layout()
plt.show()

# 2. Частота слов по темам
print("\n" + "="*60)
print("Частота слов по темам:")

# Получаем уникальные темы
topics = train_df['topic'].unique()

# Создаем словарь для хранения FreqDist по каждой теме
freq_dist_by_topic = {}

for topic in topics:
    print(f"\nТема: {topic}")

    # Фильтруем тексты по теме
    topic_texts = train_df[train_df['topic'] == topic]['processed_text']

    # Собираем все слова для данной темы
    topic_words = []
    for text in topic_texts:
        tokens = word_tokenize(text)
        topic_words.extend(tokens)

    # Создаем FreqDist для текущей темы
    freq_dist_topic = FreqDist(topic_words)
    freq_dist_by_topic[topic] = freq_dist_topic

    # Выводим топ-50 слов для темы
    top_50_topic = freq_dist_topic.most_common(50)
    print(f"Топ-50 слов в теме '{topic}':")
    for i, (word, freq) in enumerate(top_50_topic, 1):
        print(f"{i:2d}. {word}: {freq}")

    # Визуализация для каждой темы
    plt.figure(figsize=(12, 6))
    words_topic, freqs_topic = zip(*top_50_topic)
    plt.bar(range(len(words_topic)), freqs_topic)
    plt.xticks(range(len(words_topic)), words_topic, rotation=45, ha='right')
    plt.title(f'Топ-50 слов в теме: {topic}')
    plt.xlabel('Слова')
    plt.ylabel('Частота')
    plt.tight_layout()
    plt.show()

# Дополнительная информация
print("\n" + "="*60)
print("Дополнительная статистика:")
print(f"Всего уникальных слов во всей обучающей выборке: {len(freq_dist_all)}")

for topic in topics:
    freq_dist = freq_dist_by_topic[topic]
    print(f"Уникальных слов в теме '{topic}': {len(freq_dist)}")
    print(f"Общее количество слов в теме '{topic}': {freq_dist.N()}")

# Сохраняем результаты для использования в следующих заданиях
topic_word_freq = {}
for topic in topics:
    topic_word_freq[topic] = dict(freq_dist_by_topic[topic].most_common())

# Выводим несколько примеров самых характерных слов для каждой темы
print("\n" + "="*60)
print("Самые характерные слова по темам (первые 10 из топ-50):")
for topic in topics:
    top_words = list(topic_word_freq[topic].keys())[:10]
    print(f"{topic}: {', '.join(top_words)}")

**Задание 4.** Составьте словари из текстов `обучающей` части датасетов наболее часто встречающихся слов, которые встречаются **только в одной из каждых тем**. Постройте диаграмму для топ-`50` этих слов для каждой темы (ось X - слова, ось Y - частоты встречаемости слов в новостных текстах).

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict

# Создаем словари уникальных слов для каждой темы
print("Создание словарей слов, встречающихся только в одной теме...")

# Собираем все слова из всех тем
all_topic_words = defaultdict(set)
for topic in topics:
    # Берем топ-500 слов из каждой темы для анализа
    top_words = [word for word, freq in freq_dist_by_topic[topic].most_common(500)]
    all_topic_words[topic] = set(top_words)

# Находим слова, уникальные для каждой темы
unique_words_per_topic = {}

for topic in topics:
    # Слова текущей темы
    current_topic_words = all_topic_words[topic]

    # Слова всех других тем
    other_topics_words = set()
    for other_topic in topics:
        if other_topic != topic:
            other_topics_words.update(all_topic_words[other_topic])

    # Находим слова, которые есть только в текущей теме
    unique_words = current_topic_words - other_topics_words

    # Создаем словарь {слово: частота} для уникальных слов
    unique_words_freq = {}
    for word in unique_words:
        unique_words_freq[word] = freq_dist_by_topic[topic][word]

    # Сортируем по частоте и берем топ-50
    sorted_unique_words = sorted(unique_words_freq.items(), key=lambda x: x[1], reverse=True)[:50]
    unique_words_per_topic[topic] = sorted_unique_words

    print(f"\nТема: {topic}")
    print(f"Количество уникальных слов: {len(unique_words)}")
    print(f"Топ-10 уникальных слов: {[word for word, freq in sorted_unique_words[:10]]}")

# Визуализация топ-50 уникальных слов для каждой темы
for topic in topics:
    if unique_words_per_topic[topic]:
        plt.figure(figsize=(15, 8))

        words = [item[0] for item in unique_words_per_topic[topic]]
        frequencies = [item[1] for item in unique_words_per_topic[topic]]

        # Создаем столбчатую диаграмму
        bars = plt.bar(range(len(words)), frequencies, color='skyblue')

        # Настройки графика
        plt.title(f'Топ-50 уникальных слов для темы: {topic}', fontsize=16)
        plt.xlabel('Слова', fontsize=12)
        plt.ylabel('Частота встречаемости', fontsize=12)
        plt.xticks(range(len(words)), words, rotation=45, ha='right', fontsize=10)

        # Добавляем значения на столбцы
        for bar, freq in zip(bars, frequencies):
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height,
                    f'{freq}', ha='center', va='bottom', fontsize=8)

        plt.tight_layout()
        plt.show()

        # Выводим таблицу с данными
        print(f"\nТоп-50 уникальных слов для темы '{topic}':")
        df_unique = pd.DataFrame(unique_words_per_topic[topic], columns=['Слово', 'Частота'])
        print(df_unique.to_string(index=False))
    else:
        print(f"\nДля темы '{topic}' не найдено уникальных слов среди топ-500 слов")

# Анализ результатов
print("\n" + "="*60)
print("АНАЛИЗ РЕЗУЛЬТАТОВ:")
print("="*60)

for topic in topics:
    unique_count = len(unique_words_per_topic[topic])
    total_unique_words = sum([len(unique_words_per_topic[t]) for t in topics])
    print(f"Тема '{topic}': {unique_count} уникальных слов ({unique_count/total_unique_words*100:.1f}% от общего числа уникальных слов)")

# Сохраняем словари для использования в следующем задании
print("\nСохранение словарей уникальных слов...")

# Создаем отдельные словари для каждой темы
topic_vocabularies = {}
for topic in topics:
    topic_vocabularies[topic] = [word for word, freq in unique_words_per_topic[topic]]

# Выводим примеры самых характерных уникальных слов для каждой темы
print("\n" + "="*60)
print("САМЫЕ ХАРАКТЕРНЫЕ УНИКАЛЬНЫЕ СЛОВА ПО ТЕМАМ:")
print("="*60)

for topic in topics:
    if unique_words_per_topic[topic]:
        top_10_unique = [word for word, freq in unique_words_per_topic[topic][:10]]
        print(f"{topic}: {', '.join(top_10_unique)}")
    else:
        print(f"{topic}: не найдено уникальных слов")

**Задание 5.** Выполните классификацию новостных текстов из `тестовой` части датасета на основе `top-k слов` (`k` - как входной параметр) из словарей, построенных в задании `4` на основе `обучающей` части датасета. Оцените показатели `полнота` и `точность` классификации (относительно значения параметра `k`) при указании количества документов, для которых не получилось определить тему.

Общий алгоритм классификации:

- Для документа `x` считаем сколько встречается слов, из словаря каждой из тем.
- Выбираем тему для документа `x` с максимальным количеством слов
- Отдельно обрабатываем случаи, когда количество слов одинаковое или равно 0. Это случаи, когда не получилось определить тему документа.

**Задание 6.** Познакомьтесь с библиотекой Natasha для обработки текстов на русском языке, прочитав <a href="https://habr.com/ru/articles/516098/">статью</a>.

#### Импорт библиотек и создание объекта для работы с текстом
Импортируйте библиотеки для сегментации на предложения, морфологического и синтаксического анализа.

```python
from natasha import(
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    Doc,
)
```

Создайте объект из обрабатываемого текста `text`:

```python
text_doc = Doc(text)
```

#### Сегментация на предложения
Создайте объект, который будет выполнять сегментацию текста на предложения:

```python
segmenter = Segmenter()
```

С помощью сегментатора можно разбить текст на предложения и токены:

```python
text_doc.segment(segmenter)
print(text_doc.tokens)
print(text_doc.sents)
```

#### Морфологический анализ

Инициализируйте словарь и морфологический анализатор:
```python
morph_vocab = MorphVocab()
morph_tagger = NewsMorphTagger(emb)
```

Чтобы получить морфологическую разметку слова `word` воспользуйтесь:

```python
morph_vocab.parse(word)
```

Добавить морфологическую разметку к объекту обрабатываемого текста (Doc) можно как

```python
text_doc.tag_morph(morph_tagger)
print(text_doc.tokens)
text_doc.sents[0].morph.print()
```

Получить леммы текста можно через метод `lemmatize`:

```python
for token in text_doc.tokens:
    token.lemmatize(morph_vocab)
{_.text: _.lemma for _ in text_doc.tokens}
```

**Задание 7.** Добавьте столбец в обучающую и тестовую часть датасета с обработанными текстами после лемматизации.

**Задание 8.** Составьте словари на основе текстов `обучающей` части датасета по аналогии с **заданием 4.**, но с текстами, полученными после лемматзации.

**Задание 9.** Сделайте классификацию новостных текстов `тестовой` части датасета по аналогии с **заданием 5.**, но с текстами, полученными **после лемматзации**.